In [12]:
from pyserini.search import SimpleSearcher
import json

searcher = SimpleSearcher('./copa_index')
hits = searcher.search('do you always have to say check in chess?', k=3)

for i in range(len(hits)):
    content = json.loads(hits[i].raw)['contents']
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f} {content}')

 1 139  2.91510 I wrote a thank-you note to my grandmother. She became forgetful. She sent me a gift.
 2 467  2.82620 The student forgot to do her assignment. She made up an excuse to tell the teacher. The teacher promoted her to the next grade.
 3 385  2.27990 The librarian located the book in the stacks. I requested the librarian's help. The book was checked out.


In [14]:
import xml.etree.ElementTree as ET

tree = ET.parse('./data/COPA-resources/datasets/copa-test.xml')
root = tree.getroot()


In [17]:
len(root.findall('item'))

500

In [15]:
from tqdm import tqdm

kshot = 3

f = open("results/copa/text_davinci_002_bm25_{0}shot.test".format(kshot),'w')

logs = []

for item in tqdm(root.findall('item')):
    qid = item.attrib['id']
    p = item.find("p").text
    a1 = item.find("a1").text
    a2 = item.find("a2").text
    
    hits = searcher.search(p, k=kshot)
    # print(p)
    prompt = ""
    for hit in hits:
        hit = json.loads(hit.raw)
        template = "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative {3}\n\n"
        prompt += template.format(hit['data']['p'],hit['data']['1'],hit['data']['2'],hit['data']['most-plausible-alternative'])
    
    prompt += "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative".format(p,a1,a2)
    # print(prompt)
    res = generate(prompt)
    # print(res)
    
    towrite = "{0} {1} {2}".format(qid,"1" if res == "1" else "0","1" if res == "2" else "0")
    f.write(towrite+"\n")

f.close()
    
        # print(hit['data'])


100%|██████████| 500/500 [02:21<00:00,  3.53it/s]


## Random

In [16]:
tree_dev = ET.parse('./data/COPA-resources/datasets/copa-dev.xml')
root_dev = tree_dev.getroot()


In [17]:
all_docs = []

for item in root_dev.findall('item'):
    # print(item.attrib)
    doc = {
        "id":item.attrib['id'],
        "data": {
            "asks-for": item.attrib["asks-for"],
            "most-plausible-alternative": item.attrib['most-plausible-alternative'],
            "p" : item.find("p").text,
            "1": item.find("a1").text,
            "2": item.find("a2").text
        },
        "contents": "{0} {1} {2}".format(item.find("p").text, item.find("a1").text,item.find("a2").text)
    }
    all_docs.append(doc)

In [18]:
from tqdm import tqdm
import random

kshot = 3

f = open("results/copa/text_davinci_002_random_{0}shot.test".format(kshot),'w')

logs = []

for item in tqdm(root.findall('item')):
    qid = item.attrib['id']
    p = item.find("p").text
    a1 = item.find("a1").text
    a2 = item.find("a2").text
    
    # hits = searcher.search(p, k=kshot)
    hits = random.sample(all_docs, k=kshot)
    # print(p)
    prompt = ""
    
    for hit in hits:
        # hit = json.loads(hit.raw)
        template = "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative {3}\n\n"
        prompt += template.format(hit['data']['p'],hit['data']['1'],hit['data']['2'],hit['data']['most-plausible-alternative'])
    
    prompt += "Context: {0}\nAlternative 1: {1}\nAlternative 2: {2}\nCorrect: Alternative".format(p,a1,a2)
    # print(prompt)
    res = generate(prompt)
    # print(res)
    
    towrite = "{0} {1} {2}".format(qid,"1" if res == "1" else "0","1" if res == "2" else "0")
    f.write(towrite+"\n")

f.close()
    
        # print(hit['data'])


100%|██████████| 500/500 [02:16<00:00,  3.66it/s]


In [11]:
all_docs[0]

{'id': '1',
 'data': {'asks-for': 'cause',
  'most-plausible-alternative': '1',
  'p': 'My body cast a shadow over the grass.',
  '1': 'The sun was rising.',
  '2': 'The grass was cut.'},
 'contents': 'My body cast a shadow over the grass. The sun was rising. The grass was cut.'}

dynamic 0.746
predefined random: 0.666